# Pattern Recognition

## Luca Nogueira Ribeiro

### First exercise

In [43]:
%matplotlib inline
import numpy as np
import pylab as pl

mu = np.array([[3.0],[1.0],[2.0]])
mu.shape = (3,1) # Force mu to be a column vector

K = np.array([[0.5, 0.2, 0.1],
              [0.6, 0.18, 0.4],
              [0.1, 0.6, 0.2]])

x = np.array([[0],[0.5], [1]])
x.shape = (3,1)

#### Util functions

In [113]:
def getPartitions(M):
    if not isinstance(M, np.ndarray):
        raise TypeError('O argumento M deve ser do tipo numpy.ndarray')
    
    if M.shape == (3,3):
        Maa = M[0:2, 0:2]
        Mbb = M[2,2]
        Mab = M[0:2,2]
        Mba = M[2,0:2]

        # Force Mab and Mba to be column and row vectors
        Mab.shape = (2,1)
        Mba.shape = (1,2)

        return Maa, Mbb, Mab, Mba
    elif M.shape == (3,1):
        Ma = M[0:2]
        Mb = M[2]

        # Force Ma to be a column vector
        Ma.shape = (2,1)

        return Ma, Mb
    else:
        raise Exception('Argumento M deve ser uma matriz de dimensões 3x3 ou 3x1.')

def getPrecision(C):
    return np.linalg.inv(C)

def getCondCov(C):
    Caa, Cbb, Cab, Cba = getPartitions(C)
    #iCbb = np.linalg.inv(Cbb)
    iCbb = 1.0/Cbb
    return Caa - np.dot(Cab, np.dot(iCbb, Cba))

def getCondMean(C, mu, x):
    Caa, Cbb, Cab, Cba = getPartitions(C)
    #iCbb = np.linalg.inv(Cbb)
    iCbb = 1.0/Cbb
    mu_a, mu_b = getPartitions(mu)  
    xa, xb = getPartitions(x)
    aux = np.dot(np.dot(Cab, iCbb),(xb - mu_b))
    aux.shape = (2,1)
    return mu_a + aux

def getCondProb(x, C):
    xa, xb = getPartitions(x)
    ma, mb = getPartitions(mu)
    P = getPrecision(C)
    Paa, Pbb, Pab, Pba = getPartitions(P)
    
    e_aa = -0.5*np.dot((xa - ma).T, np.dot(Paa, (xa - ma)))
    e_ab = -0.5*np.dot((xa - ma).T, np.dot(Pab, (xb - mb)))
    e_ba = -0.5*np.dot((xb - mb).T, np.dot(Pba, (xa - ma)))
    e_bb = -0.5*np.dot((xb - mb).T, np.dot(Pbb, (xb - mb)))
    
    e = e_aa - e_ab - np.float(e_ba) - np.float(e_bb)
    f = (1.0/((2*np.pi)**(3.0/2.0)))*1.0/(np.sqrt(np.linalg.det(C)))
    
    return f*np.exp(e)
        
def printInfo(C):
    # Calculate the partitions of the covariance matrix
    Caa, Cbb, Cab, Cba = getPartitions(C)
    print 'Caa:'
    print Caa
    print 'Cbb:'
    print Cbb
    print 'Cab:'
    print Cab
    print 'Cba:'
    print Cba

    # Calculate the partitions of the precision matrix
    P = getPrecision(C)
    Paa, Pbb, Pab, Pba = getPartitions(P)

    # Calculate the conditional covariance matrix
    condC = getCondCov(C)
    print 'condC:'
    print condC

    # Calculate the partitions of the mean vector
    mu_a, mu_b = getPartitions(mu)
    print 'mu:'
    print mu
    print 'mu_a:'
    print mu_a
    print 'mu_b:'
    print mu_b

    # Calculate the conditional mean vector
    condMean = getCondMean(C, mu, x)
    print 'condMean:'
    print condMean

    # Calculate the probability 
    prob = getCondProb(x, C)
    print 'prob: ', prob    

In [144]:
C = np.dot(K,K.T)

print np.linalg.det(K)

xa, xb = getPartitions(x)
ma, mb = getPartitions(mu)
P = getPrecision(C)
Paa, Pbb, Pab, Pba = getPartitions(P)

e_aa = 0.5*np.dot((xa - ma).T, np.dot(Paa, (xa - ma)))
e_ab = 0.5*np.dot((xa - ma).T, np.dot(Pab, (xb - mb)))
e_ba = 0.5*np.dot((xb - mb).T, np.dot(Pba, (xa - ma)))
e_bb = 0.5*np.dot((xb - mb).T, np.dot(Pbb, (xb - mb)))


e = -np.float(e_aa) - np.float(e_ab) - np.float(e_ba) - np.float(e_bb)
f = (1.0/((2*np.pi)**(3.0/2.0)))*1.0/(np.sqrt(np.linalg.det(C)))

print e
print -0.5*np.dot((x-mu).T, np.dot(P, (x-mu)))
print np.exp(-0.5*np.dot((x-mu).T, np.dot(P, (x-mu))))


-0.0838
-80.9235109164
[[-80.92351092]]
[[  7.16746784e-36]]


#### First covariance matrix

In [120]:
# Build covariance matrix
k = 4.0
C1 = k * np.eye(3)
print 'C1:'
print C1

#printInfo(C1)
xa, xb = getPartitions(x)
ma, mb = getPartitions(mu)

print xa
print ma
print (xa-ma).T

C1:
[[ 4.  0.  0.]
 [ 0.  4.  0.]
 [ 0.  0.  4.]]
[[ 0. ]
 [ 0.5]]
[[ 3.]
 [ 1.]]
[[-3.  -0.5]]


In [111]:
# Second covariance matrix
C2 = np.dot(K, K.T)
print 'C2:'
print C2

print np.sqrt(np.linalg.det(C2))

printInfo(C2)

C2:
[[ 0.3     0.376   0.19  ]
 [ 0.376   0.5524  0.248 ]
 [ 0.19    0.248   0.41  ]]
0.0838
Caa:
[[ 0.3     0.376 ]
 [ 0.376   0.5524]]
Cbb:
0.41
Cab:
[[ 0.19 ]
 [ 0.248]]
Cba:
[[ 0.19   0.248]]
condC:
[[ 0.21195122  0.26107317]
 [ 0.26107317  0.40239024]]
mu:
[[ 3.]
 [ 1.]
 [ 2.]]
mu_a:
[[ 3.]
 [ 1.]]
mu_b:
[ 2.]
condMean:
[[ 2.53658537]
 [ 0.39512195]]
Debug:
87.8817761348
1.467474067e+38
prob:  1.11187666054e+38


In [112]:
# Third covariance matrix
C3 = np.zeros((3,3))

d = np.diag(C2)

for n in range(3):
    C3[n,n] = d[n]
    
print C3

[[ 0.3     0.      0.    ]
 [ 0.      0.5524  0.    ]
 [ 0.      0.      0.41  ]]
